    <h2><center>Supreme Court Oral Arguments Outcome Prediction Team</center></h2>
<h2 style="margin-top: -15px; margi-bottom: 10px;"><center>Final Report</center></h2>

### Project and Team Information
**Team members:** Federico Dominguez Molina, Jessup Jong, Chanteria Milner, and Michael Plunkett

**Project summary:** The project uses historic United States Supreme Court cases to train natural language processing models to predict case rulings. \
**Project repository:** [Link](https://github.com/michplunkett/supreme-court-ml-predictions) \
**Project assumptions and things to know:**
1. The number of unique roles within the advocates' file is too numerous to be helpful, so we merged them into 5 categories. While this merger may remove some variability and nuance in the file, we believe it will make it easier to derive meaningful conclusions.
    - The groupings for the roles are as follows: inferred, for respondent, for partitioner, and for amicus curiae
2. The years included within this data set are 2014 to 2019.
3. The datasets included within the previously mentioned year range are ones where the winnings side was either 0 or 1 (no missing, etc.).

**Models run:** Logistic Regression, XG Boost, and Random Forest

In [3]:
# Imports and setup

%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


### Table of Contents



### Summary Statistics

[TAKE INFORMATION FROM THE FIRST CHECKPOINT AND MAYBE PRESENT IT AS A TABLE]

### Logistic Regression

**What is a Logistic Regression model?** \
Logistic regression in machine learning is a supervised learning model that predicts the probability of a binary outcome based on training data. It is called a "supervised learning approach" because the model is trained on data where the outcome is known. 

Logistic regression is also a classification model because it predicts the probability of a binary outcome, where it predicts if the petitioner, given the 'bag of words,' won or lost the case. It is also worth noting that regularization can be added to the model to prevent overfitting. Within our current model, we decided against using that feature.

**What are some limitations of the Logistic Regression model?** \
Logistic regression is limited because it assumes a [linear relationship](https://careerfoundry.com/en/blog/data-analytics/what-is-logistic-regression/) between independent and dependent variables. It can also not consider the more complex relationships between the utterances and the Court decisions. It comes up short as the end-all-be-all model for our data since Supreme Court decisions and the speech uttered within them are not independent. Each case cannot be viewed within a vacuum but is instead influenced by precedents and the social trends of its time. Logistic regression may account for the association between verdicts and utterances. Still, it cannot explain or make any statements regarding causality.

**What were we able to conclude from our use of the Logistic Regression model?** \
We applied logistic regression models to four different datasets and observed the below results. We used the following hyperparameters as defaults in our regression: a test size of `0.20`, a random state of 123, and a maximum of 1000 iterations.

**All Utterances:** This dataset comprises a bag of words created from all utterances in the cases between 2014 and 2019, including the judge, advocate, and adversary statements. The model achieved an accuracy of `54.05%`, slightly better than random chance.

**Judge Utterances:** This dataset only focuses on a bag of words from judge statements. The model's accuracy was `52.05%`, underperforming compared to the model using all utterances.

**Advocate Utterances:** Using a bag of words created solely from advocate statements, the model achieved an accuracy of `75.67%`, significantly outperforming the model using all utterances.

**Adversary Utterances:** This dataset consists of a bag of words derived only from adversary statements. The model obtained an accuracy of `78.37%`, outperforming all other models and emerging as the best performer.

In conclusion, models using advocate and adversary utterances independently have higher predictive power than models using judge utterances or a combination of all utterances.

In [ ]:
# Code that runs the Logistic Regression model and creates visuals

### Random Forest

**What is a Random Forest model and what can you conclude from its results?** \
Random Forest is an ensemble of multiple decisions trees that uses a combination of bagging and random feature selection. Random Forest is trained through [bagging](https://link.springer.com/referenceworkentry/10.1007/978-1-4419-9863-7_612), or different bootstraps of training data, and returns the majority vote of multiple decision trees that split on a [random subset](https://link.springer.com/referenceworkentry/10.1007/978-1-4419-9863-7_602) of features. 

**What are some limitations of the Random Forest model?** \
It is harder to interpret a random forest model compared to a decision tree considering that we cannot follow through the decision process like how we would a decision tree. With bagging and random subsets of features, it takes a longer time and therefore more resources to train a random forest model.

The bag of words CountVectorizer that we used only provides predictions based on word frequency. This does not account for more complex linguistic relationship between phrases and ideas. Using a more complex vectorizer such as tf-idf could help better account for complex phrases and ideas from utterances.

**What were we able to conclude from our use of the Random Forest model?** \
We picked the random forest because we needed to capture the complex interactions among different words in unstrcutrued data, avoid overfitting by training on multiple decision trees, and rank the importance of different words on the predicted outcome. From the results, we can confirm the accuracy of the model in predicting case outcomes and see the importance of the different words. If we use this model for cross validation or out of sample data, we can also see how well the model generalizes and avoids overfitting. 

In [ ]:
# Code that runs the Random Forest model and creates visuals

### XG Boost

**What is an XG Boost model and what can you conclude from its results?** \
XG Boost, short for *Extreme Gradient Boosting* is a tree ensemble model that uses gradient boosting to minimize the loss function. It is an ensemble of multiple decision trees that uses a combination of bagging and random feature selection. XG Boost is trained through [boosting](https://link.springer.com/referenceworkentry/10.1007/978-1-4419-9863-7_612), or training on residuals, and returns the majority vote of multiple decision trees that split on a [random subset](https://link.springer.com/referenceworkentry/10.1007/978-1-4419-9863-7_602) of features.

XG Boost is an ensemble model because it uses several 'weak learners' (decision trees, in this case) to obtain a much more powerful model by adjusting the tree model on every iteration. It is different than Random Forests because, instead of averaging all of the weak learners' results to make the final predictrion, it adjusts the model on every iteration by using the previous model's residuals as the new target variable. This allows the model to learn from its mistakes and improve on every iteration.


**What are some limitations of the XG Boost model?**
[2-3 sentences]

**What were we able to conclude from our use of the Random Forest model?**
[2-3 sentences]

In [1]:
# Code that runs the XG Boost model and creates visuals

### Summation

**What model worked best for predicting case results based on utterances?**
[2-3 sentences]

**How performant were each of the models (time, processing power, etc.)?**
[2-3 sentences]

**If our constraints are time and computing power, which of the three models would be the best to implement?**
[2-3 sentences]

**What was a notable technical lesson you learned while creating this project?**

**Federico:** \
**Jessup:**
Compared to manually programming each machine learning model, I learned that using sklearn is drastically easier. This frees up a lot of time to work on optimizing the model through hyperparameters and picking amongst many available models to best fit the different kind of data we are training on. 

**Chanteria:**

**Michael:**